# 1. Data Processing

In [171]:
# import the libraries 
import numpy as np
import pandas as pd 
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp

In [172]:
# Read Data 
def read_data(file):
    with open(file, 'r') as file:
        # read first line 
        n, d, a, b = [int(x) for x in file.readline().split()] 

        # Matrix (n,d) full 0, if staff i rest day d(i) -> convert to 1 
        F = np.full((n, d), 0)  
        for staff in range(n):
            # read each line to end, [:-1] bcs end of each line is -1 
            temp = [int(x) for x in file.readline().split()[:-1]]  #check each line from 2 -> i+1
            for day in temp:
                F[staff, day-1] = 1 # day-1 bcs index of list
    return n, d, a, b, F

# Input 
n, d, a, b, F = read_data("data.txt")

In [173]:
print("Number of Staffs: {}\nNumber of Days: {}\nRange staffs of a shift: {} -> {}\nList rest day of staff i:\n{}".format(n,d,a,b,F))

Number of Staffs: 9
Number of Days: 5
Range staffs of a shift: 1 -> 3
List rest day of staff i:
[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]


# 2. CSOP

## 2.1. Optimization

In [174]:
# Declare the Model 
model = cp_model.CpModel()

# Create the Variables
# X[staff, day, shift] = 1 if staff i work on shift k of day j 
# X[staff, day, shift] = 0, otw
X = {} 
for staff in range(n):              # check each staff 
    for day in range(d):            # check each staff
        for shift in range(1,5):    # check each shift
            X[staff, day, shift] = model.NewIntVar(0,1,"X[{},{},{}]".format(staff,day,shift))

In [175]:
# Each day, a staff can only work one shift at most
for staff in range(n):    
    for day in range(d):   
        if F[staff, day] == 0:
            if day == 0:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 1)
            # If you work the night shift the day before, you can rest the next day
            else:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) + X[staff, day - 1, 4] == 1)
        else: # F[staff, day] == 1
            model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 0)

In [176]:
# Each shift in each day has at least [a] staffs and at most [b] staffs
for day in range(d):               
    for shift in range(1,5):    
        model.Add(sum([X[staff, day, shift] for staff in range(n)]) >= a)
        model.Add(sum([X[staff, day, shift] for staff in range(n)]) <= b)

In [177]:
# F(i): list of staff rest days i
# The maximum number of night shifts assigned to a specific staff is the smallest

max_night_shift = model.NewIntVar(1, int(d/2) + 1, 'max_night_shift') # limit rest day = 1/2 all days
# for loop add constraint confirm sum of all night shift of staff <= max_night_shift
for staff in range(n):
    model.Add(sum([X[staff, day, 4] for day in range(d)]) - max_night_shift <= 0)

In [178]:
# Objective Function
model.Minimize(max_night_shift)

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  print("Minimize of max night shift: ", solver.ObjectiveValue())

Minimize of max night shift:  1.0


## 2.2. Visualization

In [179]:
# Matrix (n,d,s=5) full 0, if staff i works day j, shift k -> 1 add to matrix; 0 otw
# S is work calendar of each staff 
S = np.full((n, d, 5), 0)
for staff in range(n):
    for day in range(d):
        for shift in range(1, 5):
            S[staff, day, shift] = int(solver.Value(X[staff, day, shift])) # return {0;1}

# Label days & Shift to visualize
days = np.array([f"Day {day}" for day in range(1,d+1)])
shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])

# Flat S by axis 0, use sum to return matrix include sum staff for each shift day
staff_per_shift = np.sum(S, axis=0)
# Create pandas DF to visualize
df_staff_shift = pd.DataFrame(data=staff_per_shift[:, 1:].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
df.style.background_gradient(cmap='Pastel2')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,2,3,2,1
Noon,1,2,2,1,3
Afternoon,3,2,1,2,2
Night,2,1,2,1,1


In [180]:
col = np.array([f"Staff {staff}" for staff in range(1,n+1)])
row = days
details = np.full((d,n),"Rest")

for day in range(d):
  for staff in range(n):
    for shift in range(1,5):
      if S[staff, day, shift] == 1:
        details[day,staff] = shifts[shift-1]
        break

# Check status to apply color for each type of status
def status_color(value):
  if value == "Rest": 
    color = 'Green'
  elif value == "Nigh":
    color = 'Red'
  else:
    color = 'White'
  return 'background-color: %s' % color

# Visualize details shift for each staff
pf_details = pd.DataFrame(data = details, index = row, columns = col)  
pf_details.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Morn,Noon,Afte,Morn,Morn,Afte,Nigh,Nigh,Afte
Day 2,Nigh,Noon,Morn,Morn,Afte,Afte,Rest,Rest,Noon
Day 3,Rest,Morn,Noon,Morn,Nigh,Nigh,Morn,Afte,Noon
Day 4,Afte,Rest,Morn,Nigh,Rest,Rest,Noon,Afte,Morn
Day 5,Noon,Nigh,Noon,Rest,Noon,Afte,Morn,Rest,Afte
